## 5. Kaggle - Santander Product Recommendation

**Data Description:**
In this competition, you are provided with 1.5 years of customers behavior data from Santander bank to predict what new products customers will purchase. The data starts at 2015-01-28 and has monthly records of products a customer has, such as "credit card", "savings account", etc. You will predict what additional products a customer will get in the last month, 2016-06-28, in addition to what they already have at 2016-05-28. These products are the columns named: ind_(xyz)_ult1, which are the columns #25 - #48 in the training data. You will predict what a customer will buy in addition to what they already had at 2016-05-28. 

The test and train sets are split by time, and public and private leaderboard sets are split randomly.

kaggle: https://www.kaggle.com/c/santander-product-recommendation/data

help: https://www.elenacuoco.com/2016/12/22/alternating-least-squares-als-spark-ml/?cn-reloaded=1

In [1]:
import os
import subprocess
def module(*args):        
    if isinstance(args[0], list):        
        args = args[0]        
    else:        
        args = list(args)        
    (output, error) = subprocess.Popen(['/usr/bin/modulecmd', 'python'] + args, stdout=subprocess.PIPE).communicate()
    exec(output)    
module('load', 'apps/java/jdk1.8.0_102/binary')    
os.environ['PYSPARK_PYTHON'] = os.environ['HOME'] + '/.conda/envs/jupyter-spark/bin/python'

In [2]:
import pyspark

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[2]") \
    .appName("COM6012 Collaborative Filtering RecSys") \
    .getOrCreate()

sc = spark.sparkContext

### 1. 将用户id，日期以及24个金融产品作为列名（0买，1没买）：

In [3]:
# 24 financial product
df = spark.read.csv("../Data/santander-product-recommendation/train_ver2.csv", header="true",inferSchema="true")\
.select('ncodpers','fecha_dato',
        'ind_ahor_fin_ult1', 'ind_aval_fin_ult1', 
        'ind_cco_fin_ult1', 'ind_cder_fin_ult1', 
        'ind_cno_fin_ult1', 'ind_ctju_fin_ult1', 
        'ind_ctma_fin_ult1', 'ind_ctop_fin_ult1', 
        'ind_ctpp_fin_ult1', 'ind_deco_fin_ult1', 
        'ind_deme_fin_ult1', 'ind_dela_fin_ult1', 
        'ind_ecue_fin_ult1', 'ind_fond_fin_ult1', 
        'ind_hip_fin_ult1', 'ind_plan_fin_ult1', 
        'ind_pres_fin_ult1', 'ind_reca_fin_ult1', 
        'ind_tjcr_fin_ult1', 'ind_valo_fin_ult1', 
        'ind_viv_fin_ult1', 'ind_nomina_ult1', 
        'ind_nom_pens_ult1', 'ind_recibo_ult1')


### 2. 获取日期为05-28以及06-28：

In [4]:
df_05 = df.filter(df['fecha_dato']=='2015-05-28')
df_06 = df.filter(df['fecha_dato']=='2015-06-28')

**测试集格式：**
<img src="image_github/kaggle_bank_test_csv.png" width="900" height="800">

In [5]:
lista_users= spark.read.csv("../Data/santander-product-recommendation/test_ver2.csv",header="true",inferSchema="true")

### 3. 对24个金融产品分配id： 

In [6]:
from pyspark.sql import Row
# itemcol: product id. 
ratingsRDD_06 = df_06.rdd.map(lambda p: Row(userId=p[0], itemCol=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23],rating=p[-24:]))
ratings = spark.createDataFrame(ratingsRDD_06)
ratingsRDD_05 = df_05.rdd.map(lambda p: Row(userId=p[0], itemCol=[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23],rating=p[-24:]))
val = spark.createDataFrame(ratingsRDD_05)

In [7]:
ratings.show(5) # 06

+--------------------+--------------------+---------+
|             itemCol|              rating|   userId|
+--------------------+--------------------+---------+
|[0, 1, 2, 3, 4, 5...|[0, 0, 1, 0, 0, 0...|  16132.0|
|[0, 1, 2, 3, 4, 5...|[0, 0, 1, 0, 0, 0...|1063040.0|
|[0, 1, 2, 3, 4, 5...|[0, 0, 1, 0, 0, 0...|1063041.0|
|[0, 1, 2, 3, 4, 5...|[0, 0, 1, 0, 0, 0...|1063042.0|
|[0, 1, 2, 3, 4, 5...|[0, 0, 1, 0, 0, 0...|1063043.0|
+--------------------+--------------------+---------+
only showing top 5 rows



In [8]:
val.show(5) # 05

+--------------------+--------------------+---------+
|             itemCol|              rating|   userId|
+--------------------+--------------------+---------+
|[0, 1, 2, 3, 4, 5...|[0, 0, 1, 0, 0, 0...|1061260.0|
|[0, 1, 2, 3, 4, 5...|[0, 0, 1, 0, 0, 0...|1061283.0|
|[0, 1, 2, 3, 4, 5...|[0, 0, 1, 0, 0, 0...|1061284.0|
|[0, 1, 2, 3, 4, 5...|[0, 0, 1, 0, 0, 0...|1061336.0|
|[0, 1, 2, 3, 4, 5...|[0, 0, 1, 0, 0, 0...|1061286.0|
+--------------------+--------------------+---------+
only showing top 5 rows



### 4. 产品与购买情况一一对应：

In [9]:
from pyspark.sql.functions import split, explode

ratings=ratings.select('userId', explode('itemCol'),'rating')
val=val.select('userId', explode('itemCol'),'rating')

ratings.show(5)
val.show(5)

+-------+---+--------------------+
| userId|col|              rating|
+-------+---+--------------------+
|16132.0|  0|[0, 0, 1, 0, 0, 0...|
|16132.0|  1|[0, 0, 1, 0, 0, 0...|
|16132.0|  2|[0, 0, 1, 0, 0, 0...|
|16132.0|  3|[0, 0, 1, 0, 0, 0...|
|16132.0|  4|[0, 0, 1, 0, 0, 0...|
+-------+---+--------------------+
only showing top 5 rows

+---------+---+--------------------+
|   userId|col|              rating|
+---------+---+--------------------+
|1061260.0|  0|[0, 0, 1, 0, 0, 0...|
|1061260.0|  1|[0, 0, 1, 0, 0, 0...|
|1061260.0|  2|[0, 0, 1, 0, 0, 0...|
|1061260.0|  3|[0, 0, 1, 0, 0, 0...|
|1061260.0|  4|[0, 0, 1, 0, 0, 0...|
+---------+---+--------------------+
only showing top 5 rows



In [10]:
ratingsRDD = ratings.rdd.map(lambda p: Row(userId=p[0],itemCol=p[1],ranking=p[2][int(p[1])]))
ratings2 = ratingsRDD.toDF()
ratingsRDD_val = val.rdd.map(lambda p: Row(userId=p[0],itemCol=p[1],ranking=p[2][int(p[1])]))
validation = ratingsRDD_val.toDF()

In [11]:
ratings2.show(10)
validation.show(10)

+-------+-------+-------+
|itemCol|ranking| userId|
+-------+-------+-------+
|      0|      0|16132.0|
|      1|      0|16132.0|
|      2|      1|16132.0|
|      3|      0|16132.0|
|      4|      0|16132.0|
|      5|      0|16132.0|
|      6|      0|16132.0|
|      7|      0|16132.0|
|      8|      0|16132.0|
|      9|      0|16132.0|
+-------+-------+-------+
only showing top 10 rows

+-------+-------+---------+
|itemCol|ranking|   userId|
+-------+-------+---------+
|      0|      0|1061260.0|
|      1|      0|1061260.0|
|      2|      1|1061260.0|
|      3|      0|1061260.0|
|      4|      0|1061260.0|
|      5|      0|1061260.0|
|      6|      0|1061260.0|
|      7|      0|1061260.0|
|      8|      0|1061260.0|
|      9|      0|1061260.0|
+-------+-------+---------+
only showing top 10 rows



In [12]:
training=ratings2.withColumn("userId", ratings2["userId"].cast("int"))\
.withColumn("itemCol", ratings2["itemCol"].cast("int"))\
.withColumn("ranking", ratings2["ranking"].cast("double"))

test=validation.withColumn("userId", validation["userId"].cast("int"))\
.withColumn("itemCol", validation["itemCol"].cast("int"))\
.withColumn("ranking", validation["ranking"].cast("double"))

training.show(24)

+-------+-------+------+
|itemCol|ranking|userId|
+-------+-------+------+
|      0|    0.0| 16132|
|      1|    0.0| 16132|
|      2|    1.0| 16132|
|      3|    0.0| 16132|
|      4|    0.0| 16132|
|      5|    0.0| 16132|
|      6|    0.0| 16132|
|      7|    0.0| 16132|
|      8|    0.0| 16132|
|      9|    0.0| 16132|
|     10|    0.0| 16132|
|     11|    0.0| 16132|
|     12|    0.0| 16132|
|     13|    0.0| 16132|
|     14|    0.0| 16132|
|     15|    0.0| 16132|
|     16|    0.0| 16132|
|     17|    0.0| 16132|
|     18|    0.0| 16132|
|     19|    0.0| 16132|
|     20|    0.0| 16132|
|     21|   null| 16132|
|     22|   null| 16132|
|     23|    0.0| 16132|
+-------+-------+------+
only showing top 24 rows



### 5. 对NULL进行填充：

In [13]:
training=training.na.fill(0.0)
training.show(24)

+-------+-------+------+
|itemCol|ranking|userId|
+-------+-------+------+
|      0|    0.0| 16132|
|      1|    0.0| 16132|
|      2|    1.0| 16132|
|      3|    0.0| 16132|
|      4|    0.0| 16132|
|      5|    0.0| 16132|
|      6|    0.0| 16132|
|      7|    0.0| 16132|
|      8|    0.0| 16132|
|      9|    0.0| 16132|
|     10|    0.0| 16132|
|     11|    0.0| 16132|
|     12|    0.0| 16132|
|     13|    0.0| 16132|
|     14|    0.0| 16132|
|     15|    0.0| 16132|
|     16|    0.0| 16132|
|     17|    0.0| 16132|
|     18|    0.0| 16132|
|     19|    0.0| 16132|
|     20|    0.0| 16132|
|     21|    0.0| 16132|
|     22|    0.0| 16132|
|     23|    0.0| 16132|
+-------+-------+------+
only showing top 24 rows



In [14]:
test=test.na.fill(0.0)
test.show(24)

+-------+-------+-------+
|itemCol|ranking| userId|
+-------+-------+-------+
|      0|    0.0|1061260|
|      1|    0.0|1061260|
|      2|    1.0|1061260|
|      3|    0.0|1061260|
|      4|    0.0|1061260|
|      5|    0.0|1061260|
|      6|    0.0|1061260|
|      7|    0.0|1061260|
|      8|    0.0|1061260|
|      9|    0.0|1061260|
|     10|    0.0|1061260|
|     11|    0.0|1061260|
|     12|    0.0|1061260|
|     13|    0.0|1061260|
|     14|    0.0|1061260|
|     15|    0.0|1061260|
|     16|    0.0|1061260|
|     17|    0.0|1061260|
|     18|    0.0|1061260|
|     19|    0.0|1061260|
|     20|    0.0|1061260|
|     21|    0.0|1061260|
|     22|    0.0|1061260|
|     23|    0.0|1061260|
+-------+-------+-------+
only showing top 24 rows



### 6. ALS：

In [15]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

evaluator = RegressionEvaluator(metricName="rmse", labelCol="ranking", predictionCol="prediction")

als = ALS(rank=10, maxIter=10, regParam=0.01, numUserBlocks=10, numItemBlocks=10, implicitPrefs=False, 
              alpha=1.0, 
              userCol="userId", itemCol="itemCol", seed=1, ratingCol="ranking", nonnegative=True)

model=als.fit(training)

In [16]:
topredict = test.filter(test['ranking']==0)
predictions = model.transform(topredict)
rmse = evaluator.evaluate(predictions)
rmse

nan

### 7. 推荐：

In [17]:
predictions.first()

Row(itemCol=12, ranking=0.0, userId=16339, prediction=0.09994453936815262)

In [18]:
Recommend=predictions.rdd.map(lambda p: Row(user=p[2],ProductPredictions=(p[0],p[3]))).toDF()